In [1]:
from ortools.sat.python import cp_model
import numpy as np

In [2]:
# Declare the model
model = cp_model.CpModel()

In [3]:
# Define constant variables

T = 80 # Time step number
P = 6 # h 
Column = 4
Row = 14
numNode= Row * Column # Number of nodes

#startPostions = [1,2,3,4,5,6,7,8,9,10]
startPostions = [0,1,2,3,4,5]
#R = [0,28,52,80,116,3,31,55,83,119]
R = [0,28,52,3,31,55]
goalPostions = [[R[0],R[0],R[1]],[R[1],R[1],R[2]],[R[2],R[2],R[3]],[R[3],R[3],R[4]],[R[4],R[4],R[5]],[R[5],R[5],R[0]]]
targetTimes = [[5,55,66],[5,55,65],[5,55,76],[5,55,66],[5,55,65],[5,55,76]]

In [4]:
# Define decision variables
x = [[[[model.NewIntVar(0,1,f'x_{i}_{j}_{t}_{k}') for k in range(P)] for t in range(T)] for j in range(numNode)] for i in range(numNode)]

In [5]:
# Define intermediate variables
y = [[model.NewIntVar(0,1,f'y_{k}_{t}') for t in range(T)] for k in range(P)]

In [6]:
# N_i is the set of nodes connected to node i
Map = np.arange(Row*Column).reshape(Row, Column)
N = [[] for i in range(Row*Column)]

# Four corners
'''
corners = [0,3,116,119]
N[corners[0]]=[1,4,5]
N[corners[1]]=[2,6,7]
N[corners[2]]=[112,113,117]
N[corners[3]]=[114,115,118]
'''
corners = [0,3,52,55]
N[corners[0]]=[1,4,5]
N[corners[1]]=[2,6,7]
#N[corners[2]]=[112,113,117]
N[corners[2]]=[48,49,53]
#N[corners[3]]=[114,115,118]
N[corners[3]]=[50,51,54]

# Nodes in the edge
edgeNodesB = [1,2]
edgeNodesT = [53,54]
edgeNodesL = []
edgeNodesR = []
for i in range(1,Row-1):
    edgeNodesL.append(Map[i][0])
    edgeNodesR.append(Map[i][Column-1])

for node in edgeNodesB:
    N[node]=[node,node-1,node+1,node+Column-1,node+Column,node+Column+1]

for node in edgeNodesT:
    N[node]=[node,node-1,node+1,node-Column-1,node-Column,node-Column+1]

for node in edgeNodesL:
    N[node]=[node,node-Column,node-Column+1,node+1,node+Column,node+Column+1]

for node in edgeNodesR:
    N[node]=[node,node-Column,node-Column-1,node-1,node+Column,node+Column-1]
    
# Nodes in the center part
edgeNodes = corners+edgeNodesB+edgeNodesT+edgeNodesL+edgeNodesR
nodes = list(range(Row*Column))
centerNodes = list(set(nodes)-set(edgeNodes))

for node in centerNodes:
    N[node]=[node,node-Column-1,node-Column,node-Column+1,node-1,node+1,node+Column-1,node+Column,node+Column+1]

In [7]:
# Define constraints

# Definition of x
for k in range(P):
    for t in range(T):
        model.Add(sum(x[i][j][t][k] for j in range(numNode) for i in range(numNode)) == 1)
        for i in range(numNode):
            model.Add(sum(x[i][j][t][k] for j in range(numNode) if j not in N[i]) == 0)
            #model.Add(sum(x[i][j][t][k] for j in range(numNode) if j in N[j]) <= 1)

In [8]:
for i in range(numNode):
    for t in range(T-1):
        for k in range(P):
            model.Add(sum(x[j][i][t][k] for j in range(numNode) if j in N[i]) == sum(x[i][n][t+1][k] for n in range(numNode) if n in N[i]))

In [9]:
# Collision avoidance
for j in range(numNode):
    for t in range(T):
        model.Add(sum(x[i][j][t][k] for k in range(P) for i in range(numNode) if i in N[j])<=1)

In [10]:
for t in range(T):
    for i in range(numNode):
        for j in range(numNode):
            if i!=j:
                model.Add(sum((x[i][j][t][k]+x[j][i][t][k]) for k in range(P)) <= 1)

In [11]:
# Start Postions
#for k in range(P):
#    model.Add(x[startPostions[k]][startPostions[k]][0][k]==1)
model.Add(x[startPostions[0]][startPostions[0]][0][0]==1)

In [12]:
'''
# Completion conditions
for pallet in range(P):
    for task in range(len(goalPostions[0])):
        goal = goalPostions[pallet][task]
        timing = targetTimes[pallet][task]
        model.Add(sum(x[i][goal][timing][pallet] for i in range(numNode) if i in N[goal])==1)
        for t in range(T):
            model.Add(sum(x[i][goal][t][pallet] for i in range(numNode) if i in N[goal])==1-y[pallet][t])
            if t<=timing-1:
                model.Add(-y[pallet][t]+y[pallet][t+1]<=0)
'''

'\n# Completion conditions\nfor pallet in range(P):\n    for task in range(len(goalPostions[0])):\n        goal = goalPostions[pallet][task]\n        timing = targetTimes[pallet][task]\n        model.Add(sum(x[i][goal][timing][pallet] for i in range(numNode) if i in N[goal])==1)\n        for t in range(T):\n            model.Add(sum(x[i][goal][t][pallet] for i in range(numNode) if i in N[goal])==1-y[pallet][t])\n            if t<=timing-1:\n                model.Add(-y[pallet][t]+y[pallet][t+1]<=0)\n'

In [13]:
# Objective Function
cost = sum(y[k][t] for t in range(T) for k in range(P))
model.Minimize(cost)

In [14]:
solver = cp_model.CpSolver()
status = solver.Solve(model)
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'Maximum of objective function: {solver.ObjectiveValue()}\n')
else:
    print('No solution found.')

No solution found.
